In [7]:
import gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models import CoherenceModel

## Download data

Groups full data

In [8]:
groups_df = pd.read_csv('data/groups.csv', low_memory=False)
groups_df = groups_df.drop(columns=['Unnamed: 0', 'name', 'status', 'description', 'activity', 'members_count'])

In [9]:
groups_df

,user_id,group_id
0,145,34168005
1,145,34483558
2,145,63133420
3,145,62524274
4,145,212382
...,...,...
6749302,638374890,301419447
6749303,638438887,29425083
6749304,638438887,25882042
6749305,638438887,122152852


Groups texts with nouns only

In [10]:
processed_groups = pd.read_csv('data/processed/bspb_groups_nouns.csv')
processed_groups['clear_texts'] = processed_groups['nouns_name'] + ' ' + processed_groups['nouns_description']

# Remove NaN values
processed_groups = processed_groups.loc[processed_groups['clear_texts'].notna()]

In [11]:
processed_groups

,Unnamed: 0,id,name,status,description,activity,members_count,clear_name,clear_description,nouns_name,nouns_description,clear_texts
0,0,1,ВКонтакте API,Платформа для разработчиков приложений,Группа посвящена разработке приложений на осно...,Open group,383414.0,вконтакте api,группа посвящать разработка приложение основа ...,вконтакте,группа разработка приложение основа платформа ...,вконтакте группа разработка приложение основа ...
1,1,113246211,"Сайт, видеомонтаж, портфолио для учителей, восп.","Заказ ПОРТФОЛИО, видеопоздравлений, видеопрезе...","Выполняю работы по созданию сайтов, портфолио,...",Other services,225.0,сайт видеомонтаж портфолио учитель воспа,выполнять работа создание сайт портфолио видео...,сайт видеомонтаж портфолио учитель воспа,работа создание сайт портфолио видеоролик виде...,сайт видеомонтаж портфолио учитель воспа работ...
2,2,180355074,Орешкин лес.Украшения из дерева от Александрины.,Привет.\tМеня зовут Александрина .Я создаю укр...,"Жители этого леса - украшения ,они рассказываю...",Artist,1929.0,орешкин лес украшение дерево александрина,житель лес украшение рассказывать история леге...,орешкин лес украшение дерево александрина,житель лес украшение история легенда любовь те...,орешкин лес украшение дерево александрина жите...
9,9,192937987,Гринвэй Кронштадт,NaN,☘️ GreenWay - это уникальные и доступные ЭКО -...,Household Products,105.0,гринвэй кронштадт,__u2618__ greenway это уникальный доступный эк...,гринвэй кронштадт,эко продукт жизнь помощь нанотехнология чистот...,гринвэй кронштадт эко продукт жизнь помощь нан...
11,11,14,Дом Культуры,Дом культуры,Анонсы культурных событий Петербурга (и немнож...,Community Center,5991.0,дом культура,анонс культурный событие петербург немножко мо...,дом культура,анонс событие петербург москва,дом культура анонс событие петербург москва
...,...,...,...,...,...,...,...,...,...,...,...,...
912471,912471,64449308,Музейный центр Ваприикки | Тампере | Финляндия,Музей | Сайт музея на русском языке: www.vapri...,Музейный центр Ваприикки в центре города Тампе...,"Museum, gallery, exhibition",183.0,музейный центр ваприикки тампере финляндия,музейный центр ваприикки центр город тампере п...,центр ваприикки тампере финляндия,центр ваприикки центр город тампере семья биле...,центр ваприикки тампере финляндия центр ваприи...
912476,912476,104857566,"ПРОДАЖА,ПОКУПКА и ОБМЕН АВТОМОБИЛЕЙ .(AUTOHOUSE)",Тел. 8-953-231-48-00,"- Оформление, договора купли-продажи в любое в...",Transport,135.0,продажа покупка обмен автомобиль autohouse,оформление договор купля продажа любой время п...,продажа покупка обмен автомобиль,оформление договор купля продажа время продажа...,продажа покупка обмен автомобиль оформление до...
912480,912480,83908177,Продукты с фермы,Фермерские продукты,Натуральные продукты с фермы\n\nМы обеспечивае...,Internet store,847.0,продукт ферма,натуральный продукт ферма обеспечивать свой кл...,продукт ферма,продукт ферма клиент продукция сыр сулугуни сы...,продукт ферма продукт ферма клиент продукция с...
912485,912485,54525929,Байконур | Новости | Объявления - 94info,Все объявления Байконура в одном месте!,Для размещения объявления в группе нажимаем ''...,City community,47036.0,байконур новость объявление,размещение объявление группа нажимать __u0001f...,байконур новость объявление,размещение объявление группа запись правило ре...,байконур новость объявление размещение объявле...


Posts and reposts

In [12]:
processed_reposts = pd.read_csv('data/processed/bspb_reposts_nouns.csv')
processed_posts = pd.read_csv('data/processed/bspb_posts_nouns.csv')

# Remove NaN values
processed_reposts = processed_reposts.loc[processed_reposts['nouns'].notna()]
processed_posts = processed_posts.loc[processed_posts['nouns'].notna()]

In [13]:
processed_reposts

,Unnamed: 0,owner_id,post_id,text,clear_text,nouns
0,0,273694109,826,РОССИЯ: ОСТАНОВИТЬ ЗАКОНОПРОЕКТ О ПРИНУДИТЕЛЬН...,россия останавливать законопроект принудительн...,россия законопроект вакцинация ноябрь гос дума...
1,1,116524602,10482,Спиcoк caмых эффeктивных лeкaрcтв нa рaзныe cл...,список самый эффективный лекарство разный случ...,список лекарство случай жизнь
2,2,15835662,1127,"Кому меда? еще остался, прямиком из Тайги.\nЗа...",мед оставаться прямиком тайга качество ручатьс...,мед тайга качество личка
3,3,31868444,6676,22 МАЯ | 20:00\nТанцевальный спектакль «плач/ь...,май танцевальный спектакль плач ь билет dancek...,май спектакль плач билет счастие счастие свобо...
4,4,31868444,6673,"Сегодня день рождения [id31868444|Даши], нашег...",сегодня день рождение даша наш фитнес тренер д...,день рождение даша фитнес тренер растяжка утро...
...,...,...,...,...,...,...
6390,6390,554424,5189,Давайте спасать планету!,давать спасать планета,планета
6393,6393,1824199,3202,Подарочный сертификат🏃‍🏃‍🏃‍\n\n22 декабря – ка...,подарочный сертификат __u0001f3c3__ __u0001f3c...,сертификат декабрь катастрофа город сувенир па...
6394,6394,267284787,396,Учись студент,учиться студент,студент
6395,6395,17665225,1333,Беда не приходит одна 😆,беда приходить __u0001f606__,беда


In [14]:
processed_posts

,Unnamed: 0,from_id,post_id,text,clear_text,nouns
1,1,145,2292,"Сколько же всего изменилось за это время, 7 ле...",сколько изменяться время год проходить рождать...,время год сын бизнес вещь челси сердце клуб иг...
2,2,145,2292,"Жаль, что из-за политики Абрамовича клуб так и...",жаль политика абрамович клуб начинать играть с...,политика абрамович клуб надежда челси слава
3,3,145,2290,Полтора года назад искал решение некоторых сво...,полтора год назад искать решение некоторый сво...,год решение проблема простор интернет виктор п...
4,4,145,2290,На данный момент дружим семьями и безумно благ...,данный момент дружить семья безумно благодарны...,момент семья судьба человек
5,5,145,2290,К сожалению не имею возможности поздравить лич...,сожаление иметь возможность поздравлять лично ...,сожаление возможность год жизнь ребенок успех
...,...,...,...,...,...,...
810835,810835,619951213,49,она вместит себе в глотку револьвер,вмещать глотка револьвер,глотка револьвер
810836,810836,619951213,47,огонь да мясо,огонь мясо,огонь мясо
810837,810837,619951213,35,Твой аромат.,твой аромат,аромат
810838,810838,620178523,449,Вводи код 6s2jq8 и получай 5 руб!,вводить код получать руб,код руб


## Analyze users

In [17]:
lda_model_groups = gensim.models.ldamodel.LdaModel.load('models/lda_model_groups_notna_13.model')
lda_model_posts_reposts = gensim.models.ldamodel.LdaModel.load('models/lda_model_posts_reposts_nouns_notna_12.model')

In [18]:
# An auxiliary function for getting corpus from DF
def get_corpus(series):
    texts_list = [str(values).split() for values in series]
    id2word = corpora.Dictionary(texts_list)
    corpus = [id2word.doc2bow(text) for text in texts_list]
    return corpus

In [19]:
# Function for getting vectors of user's activities and opinions (posts and reposts) and interests (groups)
def analyze_user(user_id):
    
    user_posts = processed_posts.loc[processed_posts['from_id'] == int(user_id)]
    user_reposts = processed_reposts.loc[processed_reposts['owner_id'] == int(user_id)]
    user_posts_reposts = pd.concat([user_posts['nouns'], user_reposts['nouns']])
    if len(user_posts_reposts) > 0:
        posts_reposts_corpus = get_corpus(user_posts_reposts)
        lda_model_activities = lda_model_posts_reposts
        posts_reposts_topics = lda_model_activities[posts_reposts_corpus]
        posts_reposts_topics = posts_reposts_topics[0]
        
    else:
        posts_reposts_topics = [(0, None),
                                (1, None),
                                (2, None),
                                (3, None),
                                (4, None),
                                (5, None),
                                (6, None),
                                (7, None),
                                (8, None),
                                (9, None),
                                (10, None),
                                (11, None)]
    
    user_groups_list = list(groups_df.loc[groups_df['user_id'] == str(user_id)]['group_id'].values)
    user_groups_list = [int(group) for group in user_groups_list]
    user_groups = processed_groups.loc[processed_groups['id'].isin(user_groups_list)]
    if len(user_groups) > 0:    
        groups_corpus = get_corpus(user_groups['clear_texts'])
        lda_model_interests = lda_model_groups
        groups_topics = lda_model_interests[groups_corpus]
        groups_topics = groups_topics[0]
    else:
        groups_topics = [(0, None),
                                (1, None),
                                (2, None),
                                (3, None),
                                (4, None),
                                (5, None),
                                (6, None),
                                (7, None),
                                (8, None),
                                (9, None),
                                (10, None),
                                (11, None),
                                (12, None)]
    return posts_reposts_topics, groups_topics

In [20]:
analyze_user(17039363)[1][0]

[(0, 0.020074721),
 (1, 0.019182287),
 (2, 0.30670038),
 (4, 0.5829292),
 (9, 0.018949088),
 (12, 0.036696296)]

### Collect set of users

In [21]:
users_with_posts = set(processed_posts['from_id'])

# Convert ids into int
users_with_posts = [int(user_id) for user_id in users_with_posts]
len(users_with_posts)

12546

In [22]:
users_with_reposts = set(processed_reposts['owner_id'])

# Convert ids into int
users_with_reposts = [int(user_id) for user_id in users_with_reposts]
len(users_with_reposts)

2580

In [38]:
users_with_groups = set(groups_df['user_id'])
users_with_groups = list(users_with_groups)

# Remove nan
users_with_groups = users_with_groups[1:]
# Remove user_id
user_id_ind = users_with_groups.index('user_id')
users_with_groups.pop(10228)

# Convert ids into int
users_with_groups = [int(user_id) for user_id in users_with_groups]
len(users_with_groups)

28970

Final set of users

In [39]:
users = (set(users_with_posts).union(set(users_with_reposts))).union(set(users_with_groups))
len(users)

28997

In [271]:
topics_posts_reposts = []
topics_groups = []

In [ ]:
for u in users:  
    t_posts_reposts, t_groups = analyze_user(u)
    topics_posts_reposts.append(t_posts_reposts)
    topics_groups.append(t_groups)

In [282]:
POSTS_REPOSTS_MODEL_TOPICS_NUM = 12
GROUPS_MODEL_TOPICS_NUM = 13

In [285]:
final_df = pd.DataFrame()
final_df['user_id'] = users

for topic_n in range(POSTS_REPOSTS_MODEL_TOPICS_NUM):
    tops_n = []
    for topic in topics_posts_reposts:
        flag = False
        
        if len(topic) != POSTS_REPOSTS_MODEL_TOPICS_NUM:
            topic = topic[0]
        for item in topic:
            if item[0] == topic_n:
                tops_n.append(item[1])
                flag = True
        
        if not flag:
            tops_n.append(None)

    final_df[f'posts_reposts_{topic_n}'] = tops_n

for topic_n in range(GROUPS_MODEL_TOPICS_NUM):
    tops_n = []
    for topic in topics_groups:
        flag = False
        
        if len(topic) != GROUPS_MODEL_TOPICS_NUM:
            topic = topic[0]
        for item in topic:
            if item[0] == topic_n:
                tops_n.append(item[1])
                flag = True
        
        if not flag:
            tops_n.append(None)

    final_df[f'groups_{topic_n}'] = tops_n

In [286]:
final_df

,user_id,posts_reposts_0,posts_reposts_1,posts_reposts_2,posts_reposts_3,posts_reposts_4,posts_reposts_5,posts_reposts_6,posts_reposts_7,posts_reposts_8,...,groups_3,groups_4,groups_5,groups_6,groups_7,groups_8,groups_9,groups_10,groups_11,groups_12
0,17489295,0.053998,0.040320,0.061113,0.062024,0.053151,0.067862,0.314564,0.059733,0.077446,...,NaN,0.456763,NaN,NaN,NaN,0.017124,NaN,NaN,NaN,NaN
1,91020,0.058680,0.043816,0.066413,0.067403,0.057760,0.073747,0.255121,0.064913,0.084162,...,NaN,0.629385,NaN,NaN,NaN,NaN,0.041771,NaN,NaN,NaN
2,17039363,0.064255,0.047979,0.072723,0.073806,0.063248,0.080753,0.184357,0.071080,0.092158,...,NaN,0.581425,NaN,NaN,NaN,NaN,0.018878,NaN,NaN,0.038319
3,2430764,0.064255,0.047979,0.072723,0.073806,0.063248,0.080753,0.184357,0.071080,0.092158,...,NaN,0.647599,NaN,NaN,NaN,0.011934,NaN,NaN,NaN,NaN
4,76546053,0.064255,0.047979,0.072723,0.073806,0.063248,0.080753,0.184357,0.071080,0.092158,...,0.064162,0.354159,NaN,NaN,NaN,NaN,0.049147,NaN,NaN,0.058909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41631,177864684,0.053998,0.040320,0.061113,0.062024,0.053151,0.067862,0.314564,0.059733,0.077446,...,0.014069,0.296296,0.012418,0.013902,0.013982,0.030362,0.017184,NaN,0.017058,0.016408
41632,18117179,0.064255,0.047979,0.072723,0.073806,0.063248,0.080753,0.184357,0.071080,0.092158,...,0.020940,0.029172,0.018496,0.020703,0.020822,0.044191,0.025595,0.014819,0.025332,0.024431
41633,24248311,0.064255,0.047979,0.072723,0.073806,0.063248,0.080753,0.184357,0.071080,0.092158,...,NaN,0.647638,NaN,NaN,NaN,0.011934,NaN,NaN,NaN,NaN
41634,1216919,0.064255,0.047979,0.072723,0.073806,0.063248,0.080753,0.184357,0.071080,0.092158,...,NaN,0.662201,NaN,NaN,NaN,0.012472,NaN,NaN,NaN,NaN


In [287]:
final_df.to_csv('users_activities_groups.csv')